In [4]:
# 读取task集合  Id packFormat packUrl productName categoryCode categoryName isFinished
# 判断packFormat是否为pdf
# 记住Id，下载文档到dir_input
# 分割到dir_segmentation
# 识别结果为字典，框框图片保存到dir_recognition
# 识别结果格式重组+sPicture

# 识别结果加入到result集合
# task集合 isFinished更新

# Id数据类型为double？
# log 

In [1]:
import os
import sys
import pathlib
import glob
import torch
import numpy
from natsort import natsorted
from PdfMake import PDFMake#
from predict import recog_label
from tqdm import tqdm
import pymongo
import requests
import copy
import datetime

# 连接
database_name = 'test'
def connect_to_db(host, database_name):
    client = pymongo.MongoClient(host=host)
    print(client.list_database_names())
    db = client[database_name]
    print(db)
    return client, db

client,db = connect_to_db('localhost', 'test')

['admin', 'config', 'local', 'test']
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test')


In [2]:
def get_file_list(folder_path: str, p_postfix: list = None, sub: bool = True) -> list:
    assert os.path.exists(folder_path) and os.path.isdir(folder_path)
    if p_postfix is None:
        p_postfix = ['.pdf']
    if isinstance(p_postfix, str):
        p_postfix = [p_postfix]
    file_list = [x for x in glob.glob(folder_path + '/**/*.*', recursive=True) if
                 os.path.splitext(x)[-1] in p_postfix or '.*' in p_postfix]
    return natsorted(file_list)

def download_from_url(url,name):
    print("downloading with requests")
    #url = 'http://gateway.ntdjk.com/blade-resource/oss/endpoint/file-view-link?attachId=1420763982380843009'
    r = requests.get(url)
    #print(r)
    with open(name+".pdf", "wb") as code:
        code.write(r.content)
        print('finished')
        
## 下载
def download_from_task(task, saved_dir):
    download_url = task['packUrl']
    # saved_dir = dir_input
    pdf_name = str(task['Id']) + task['productName']
    # 创建专属目录 dir_input/pdf_name/
    special_dir = saved_dir + pdf_name + '/'
    if not os.path.exists(special_dir):
        os.makedirs(special_dir)
    
    print(special_dir)
    print(pdf_name)

    download_from_url(download_url, special_dir+pdf_name)
    return special_dir

## 分割
def segmentation_pdf(input_dir, output_dir):
    for pdf in tqdm(get_file_list(input_dir, p_postfix=['.pdf'])):
        file_name = os.path.basename(pdf).split('.')[0]
        print(file_name)
        if not os.path.exists(output_dir+file_name):
            os.makedirs(output_dir+file_name)
        master = PDFMake(outputdirectory=output_dir+file_name)
        master.spilt_pdf(pdf, 1, name = file_name)
        print('segmentation file name:',file_name)
    return output_dir+file_name
        
        
## 识别
dict_result = {'商标': 'True', '经销企业': 'True', '不适宜人群': 'True', '许可证编号': 'True', '执行标准': 'True', '条形码': 'True',
                   '保健品商标': 'True', '成分': 'True', '功效': 'True', 'r标': 'True'}
def recognition_img(input_dir, output_dir):
    for img in tqdm(get_file_list(input_dir, p_postfix=['.jpg'])):
        file_name = os.path.basename(img).split('.')[0]
        print(file_name)
        if not os.path.exists(output_dir+file_name):
            os.makedirs(output_dir+file_name)
        final_result = recog_label(img, output_dir+file_name, dict_result, file_name)
        return final_result
        #final_results.append(final_result)
        #print(file_name)
        
# 重组字典，写入result
def insert_result(final_result, task, to_collection):
    final_one = {}
    for index in final_result:
        if index == 'name':
            pass
        else:
            final_one['taskId'] = task['Id']
            final_one['auditName'] = index
            final_one['scanFount'] = final_result[index][0]
            final_one['scanContent'] = final_result[index][1]
            print(final_one)
            #=========================
            temp = copy.deepcopy(final_one)
            result = to_collection.insert_one(temp)
            
            
# 更新状态
def task_is_finished(Id, collection='tasks', state = True):
    old_value_id = {"Id": Id}
    new_value = {"$set": {"isFinished": state}}
    db[collection].update_one(old_value_id, new_value)

In [6]:
# 循环操作tasks
dir_seg = './dir_segmentation/'
dir_input = './dir_input/'
dir_rec = './dir_recognition/'
finish_flag = False
# if 'task' do not have a packUrl?, not a pdf?
def operation_id(task_collection_name, id_number):
    tasks_collection = db[task_collection_name]
    for task in tasks_collection.find({'Id':id_number}):
        time1 = datetime.datetime.now()
        download_url = task['packUrl']
        print('processing:', download_url)
        # download
        special_dir = download_from_task(task, dir_input)
        time2 = datetime.datetime.now()
        # segmentation
        special_dir = segmentation_pdf(special_dir, dir_seg)
        # recognition
        final_result = recognition_img(special_dir, dir_rec)
        time3 = datetime.datetime.now()
        # insert
        insert_result(final_result, task, db.results)
        # update
        task_is_finished(task['Id'], collection='tasks', state = True)

        print('download time:', (time2 - time1).seconds)
        print('recognition time:', (time3 - time2).seconds)
        break
        #print(task['isFinished'])
    else:
        pass
    
def operation_from_collection():
    tasks_collection = db['tasks']

    for task in tasks_collection.find():
        # if 'task' do not have a packUrl?, not a pdf?
        if task['packFormat'] == 'pdf':
        #and task['isFinished'] == False:
           # print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            time1 = datetime.datetime.now()
            download_url = task['packUrl']
            print('processing:', download_url)
            # download
            special_dir = download_from_task(task, dir_input)
            time2 = datetime.datetime.now()
            # segmentation
            special_dir = segmentation_pdf(special_dir, dir_seg)
            # recognition
            final_result = recognition_img(special_dir, dir_rec)
            time3 = datetime.datetime.now()
            # insert
            insert_result(final_result, task, db.results)
            # update
            task_is_finished(task['Id'], collection='tasks', state = True)
            
            print('download time:', (time2 - time1).seconds)
            print('recognition time:', (time3 - time2).seconds)
            break
            #print(task['isFinished'])
        else:
            pass
            
            
#operation_from_collection()
operation_id('tasks', 111111)

processing: http://gateway.ntdjk.com/blade-resource/oss/endpoint/file-view-link?attachId=1420763982380843009
./dir_input/111111yaowu1/
111111yaowu1
downloading with requests
finished


  0%|          | 0/1 [00:00<?, ?it/s]

111111yaowu1


100%|██████████| 1/1 [00:02<00:00,  2.52s/it]


segmentation file name: 111111yaowu1


  0%|          | 0/1 [00:00<?, ?it/s]

111111yaowu11


[2022-01-14 15:32:21,752] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object
[2022-01-14 15:32:23,789] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object
  0%|          | 0/1 [00:56<?, ?it/s]

{'taskId': 111111, 'auditName': '商标', 'scanFount': 'True', 'scanContent': '国南京同仁堂'}
{'taskId': 111111, 'auditName': '经销企业', 'scanFount': 'False', 'scanContent': 'False'}
{'taskId': 111111, 'auditName': '不适宜人群', 'scanFount': 'False', 'scanContent': 'False'}
{'taskId': 111111, 'auditName': '许可证编号', 'scanFount': 'False', 'scanContent': 'False'}
{'taskId': 111111, 'auditName': '执行标准', 'scanFount': 'False', 'scanContent': 'False'}
{'taskId': 111111, 'auditName': '条形码', 'scanFount': 'True', 'scanContent': '6970879610928'}
{'taskId': 111111, 'auditName': '保健品商标', 'scanFount': 'True', 'scanContent': 'True'}
{'taskId': 111111, 'auditName': '成分', 'scanFount': 'False', 'scanContent': 'False'}
{'taskId': 111111, 'auditName': '功效', 'scanFount': 'True', 'scanContent': '唯护美白祛斑面膜'}
{'taskId': 111111, 'auditName': 'r标', 'scanFount': 'True', 'scanContent': 'R标'}
download time: 11
recognition time: 59


In [30]:
addone = {
    'Id':123456789,
    'packFormat':'pdf',
    'packUrl':'http://gateway.ntdjk.com/blade-resource/oss/endpoint/file-view-link?attachId=1419848206056615938',
    'productName':'投保',
    'isFinished': False
}
db.tasks.insert_one(addone)